# Ridge

In [2]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_theme(
    context="paper", 
    style="whitegrid", 
    font_scale=1.2,
    rc={'figure.figsize': (10, 10), 'figure.dpi': 300}
)

## Get Data

In [3]:
from common.data import get_data
from common.paths import HEALTHY, ADHD

X, Y, demographics, population = get_data(label_path=ADHD)
ages, sexes = demographics['Age'], demographics['Sex']

print(f'X: {X.shape} | Y: {len(Y.keys())} | Age: {ages.shape} | Population: {population}')

X: (373, 34716) | Y: 6 | Age: (373,) | Population: adhd


In [7]:
selected_target = "WISC_FSIQ"
y = Y[selected_target]

print(f'y: {y.shape}')

y: (373,)


In [8]:
from common.binning import bin_by_age

bins = bin_by_age(X, y, ages)
bin_1, bin_2, bin_3 = bins[0], bins[1], bins[2]
print(f'Bin 1: {bin_1[0].shape} | Bin 2: {bin_2[0].shape} | Bin 3: {bin_3[0].shape}')

Bin 1: (114, 34716) | Bin 2: (147, 34716) | Bin 3: (112, 34716)


In [8]:
bin_2_subsample = np.random.choice(152, 114, replace=False)
bin_2_ss = (bin_2[0][bin_2_subsample], bin_2[1][bin_2_subsample])

print(bin_2_ss[0].shape, bin_2_ss[1].shape)

(114, 34716) (114,)


## Run for one target, one age bin

In [4]:
from common.scoring import unimetric_scorer, multimetric_scorer, N_PERM, SCORING, RKF_10_10, RKF_5_5
from sklearn.linear_model import Ridge, RidgeCV, RidgeClassifier
from sklearn.model_selection import cross_validate, permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

### Set up the model pipeline, metrics, and cross-validation approach

In [10]:
%%time
X_cv = bin_2_ss[0]
y_cv = bin_2_ss[1]
# X_cv = X
# y_cv = y
age_group = 'Bin 2'

estimators = [StandardScaler(), RidgeCV(alphas=[a for a in range(1, 10000, 100)], 
                                        scoring=unimetric_scorer, cv=RKF_10_10)]
pipe = make_pipeline(*estimators)
pipe.fit(X_cv, y_cv)
ridge_cv = pipe['ridgecv']

print(f'Target: {selected_target} | Alpha: {ridge_cv.alpha_} | Score: {ridge_cv.best_score_:.2f}')

Target: WISC_FSIQ | Alpha: 9901 | Score: 0.29
CPU times: user 19min 9s, sys: 17.8 s, total: 19min 27s
Wall time: 1min 56s


### Run cross-validation

In [9]:
%%time
estimators = [StandardScaler(), Ridge(alpha=ridge_cv.alpha_)]
pipe = make_pipeline(*estimators)

scores = cross_validate(pipe, X_cv, y_cv, cv=RKF_10_10, scoring=unimetric_scorer, n_jobs=-1, 
                        return_train_score=True, return_estimator=True)

coefs = np.array([estimator['ridge'].coef_ for estimator in scores['estimator']])
avg_coef = np.mean(coefs, axis=0)
intercepts = np.array([estimator['ridge'].intercept_ for estimator in scores['estimator']])
avg_inte = np.mean(intercepts, axis=0)

print(f'ridge_{population}_{selected_target}_{age_group}')
print(f'Avg test score: {np.mean(scores["test_score"])}')

ridge_adhd_WISC_PSI_Bin 2
Avg test score: 0.64697580295615
CPU times: user 570 ms, sys: 280 ms, total: 850 ms
Wall time: 2.47 s


### Save Model Weights

In [38]:
from os.path import join
from common.paths import RIDGE_WEIGHTS

coef_fn = f'ridge_{population}_{selected_target}_{age_group}_coef.npy'
inte_fn = f'ridge_{population}_{selected_target}_{age_group}_inte.npy'

np.save(join(RIDGE_WEIGHTS, coef_fn), avg_coef)
np.save(join(RIDGE_WEIGHTS, inte_fn), avg_inte)

### ICC

In [34]:
%%time
import pingouin as pg

icc_data = pd.DataFrame(coefs).melt(var_name='connection', value_name='weight', ignore_index=False)
icc_data['cv_run_num'] = icc_data.index
icc = pg.intraclass_corr(data=icc_data, targets='connection', raters='cv_run_num', ratings='weight').round(3)
icc.set_index("Type")

,Description,ICC,F,df1,df2,pval,CI95%
Type,,,,,,,
ICC1,Single raters absolute,0.897,873.76,34715,3436884,0.0,"[0.9, 0.9]"
ICC2,Single random raters,0.897,873.88,34715,3436785,0.0,"[0.9, 0.9]"
ICC3,Single fixed raters,0.897,873.88,34715,3436785,0.0,"[0.9, 0.9]"
ICC1k,Average raters absolute,0.999,873.76,34715,3436884,0.0,"[1.0, 1.0]"
ICC2k,Average random raters,0.999,873.88,34715,3436785,0.0,"[1.0, 1.0]"
ICC3k,Average fixed raters,0.999,873.88,34715,3436785,0.0,"[1.0, 1.0]"


### Run permutation test statistic (pts)

In [11]:
%%time
estimators = [StandardScaler(), Ridge(alpha=ridge_cv.alpha_)]
pipe = make_pipeline(*estimators)

score, _, pvalue = permutation_test_score(
    pipe, X_cv, y_cv, cv=RKF_10_10, scoring=unimetric_scorer, n_permutations=N_PERM, n_jobs=-1)

print(f'ridge_{population}_{selected_target}_{age_group}')
print(f'Alpha: {ridge_cv.alpha_} | Score: {score:.2f} | p-value: {pvalue:.4f}')

ridge_adhd_WISC_FSIQ_Bin 2
Alpha: 9901 | Score: 0.33 | p-value: 0.0100
CPU times: user 56.9 s, sys: 1.5 s, total: 58.4 s
Wall time: 15min 57s


## Run for all targets, all age bins

### Run cross-validation

In [53]:
%%time
from os.path import join
from common.binning import bin_data
from common.wisc import FSIQ, PRIMARY_INDICES
from common.paths import RIDGE_WEIGHTS, RIDGE_RESULTS

results = []
targets = FSIQ + PRIMARY_INDICES

for target in targets:
    y = Y[target]
    X_all, y_all, bin_labels = bin_data(X, y)
    
    for X_cv, y_cv, bin_label in zip(X_all, y_all, bin_labels):
        # Find best alpha
        estimators = [StandardScaler(), RidgeCV(alphas=[a for a in range(5000, 75000, 5000)], 
                                        scoring=unimetric_scorer, cv=RKF_10_10)]
        pipe = make_pipeline(*estimators).fit(X_cv, y_cv)
        ridge_cv = pipe['ridgecv']
        
        # Do cross-validation
        estimators = [StandardScaler(), Ridge(alpha=ridge_cv.alpha_)]
        pipe = make_pipeline(*estimators)
        
        scores = cross_validate(pipe, X_cv, y_cv, cv=RKF_10_10, 
                                scoring=unimetric_scorer, n_jobs=-1, 
                                return_train_score=False, 
                                return_estimator=True)
        coefs = np.array([estimator['ridge'].coef_ for estimator in scores['estimator']])
        avg_coef = np.mean(coefs, axis=0)
        intercepts = np.array([estimator['ridge'].intercept_ for estimator in scores['estimator']])
        avg_inte = np.mean(intercepts, axis=0)
        
        results.append({    
            'Model': 'ridge',
            'Population': population,
            'Target': target,
            'Bin': bin_label,
            'Alpha': ridge_cv.alpha_,
            'Score': np.mean(scores['test_score']),
        })
        print(results[-1])
        
        coef_fn = f'ridge_{population}_{target}_{bin_label}_coef.npy'
        inte_fn = f'ridge_{population}_{target}_{bin_label}_inte.npy'
        np.save(join(RIDGE_WEIGHTS, coef_fn), avg_coef)
        np.save(join(RIDGE_WEIGHTS, inte_fn), avg_inte)

results_df = pd.DataFrame(results)
display(results_df.round(4))
filename = f'ridge_cv_{population}.csv'
results_df.to_csv(join(RIDGE_RESULTS, filename))

KeyboardInterrupt: 

### Run permutation test statistic (pts)

In [ ]:
%%time
from os.path import join
from common.binning import bin_data
from common.wisc import FSIQ, PRIMARY_INDICES
from common.paths import RIDGE_RESULTS

results = []
targets = FSIQ + PRIMARY_INDICES

# bin_2_subsample = np.random.choice(152, 114, replace=False)  # TEMP

for target in targets:
    y = Y[target]
    X_all, y_all, bin_labels = bin_data(X, y, ages)
#     X_all[1], y_all[1] = X_all[1][bin_2_subsample], y_all[1][bin_2_subsample]  # TEMP
    
    for X_cv, y_cv, bin_label in zip(X_all, y_all, bin_labels):
#         if bin_label != "Bin 2":  # TEMP
#             continue
        
        # Find best alpha
        estimators = [StandardScaler(), RidgeCV(alphas=[a for a in range(1, 10000, 100)], 
                                        scoring=unimetric_scorer, cv=RKF_5_5)]
        pipe = make_pipeline(*estimators).fit(X_cv, y_cv)
        ridge_cv = pipe['ridgecv']
        
        # Do permutation test
        estimators = [StandardScaler(), Ridge(alpha=ridge_cv.alpha_)]
        pipe = make_pipeline(*estimators)
        
        score, permutation_scores, pvalue = permutation_test_score(
            pipe, X_cv, y_cv, cv=RKF_5_5, scoring=unimetric_scorer, n_permutations=N_PERM, 
            n_jobs=-1)
        results.append({    
            'Model': 'ridge',
            'Population': population,
            'Target': target,
            'Bin': bin_label,
            'Alpha': ridge_cv.alpha_,
            'Score': score,
            'P-value': pvalue,
        })
        print(results[-1])
        
results_df = pd.DataFrame(results)
display(results_df.round(4))
filename = f'ridge_pts_{population}.csv'
results_df.to_csv(join(RIDGE_RESULTS, filename))

{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'Bin': 'Bin 1', 'Alpha': 1, 'Score': 0.20985556337736747, 'P-value': 0.03992015968063872}
{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'Bin': 'Bin 2', 'Alpha': 2401, 'Score': 0.34620853517798267, 'P-value': 0.001996007984031936}
{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'Bin': 'Bin 3', 'Alpha': 1401, 'Score': 0.04896150402257205, 'P-value': 0.30538922155688625}
{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_VSI', 'Bin': 'Bin 1', 'Alpha': 7001, 'Score': 0.25469209948287497, 'P-value': 0.00998003992015968}
{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_VSI', 'Bin': 'Bin 2', 'Alpha': 9901, 'Score': 0.19918578470603093, 'P-value': 0.01996007984031936}
{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_VSI', 'Bin': 'Bin 3', 'Alpha': 9901, 'Score': 0.047888935846392304, 'P-value': 0.3473053892215569}
{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_VCI', 